<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
import spacy

nlp = spacy.load('en_core_web_lg')

In [4]:
# defining stop_words

stop_words = nlp.Defaults.stop_words

In [83]:
# function to do tokenization on test

def tokenize(docs):
    for tokens in docs:
        
        text = [tokens.lemma_ for tokens in nlp(docs) if
               (tokens.is_punct == False) and
               (tokens.text.lower() not in stop_words)]
        
    return text
    

In [6]:
# import progress apply function

import pandas as pd
from tqdm import tqdm
tqdm.pandas()

C:\Users\Samsung\Anaconda3\envs\unit4-NLP\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [84]:
# testing to make sure function works. 

test_run = yelp['text'][:5].progress_apply(tokenize)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:48<00:00,  9.75s/it]


In [85]:
test_run

0    [beware, fake, fake, fake, small, business, Lo...
1    [come, lunch, Togo, service, quick, staff, fri...
2    [Vegas, dozen, time, step, foot, Circus, Circu...
3    [go, night, close, street, party, good, actual...
4    [3.5, 4, star, \n\n, bad, price, $, 12.99, lun...
Name: text, dtype: object

In [86]:
test_run[0]

['beware',
 'fake',
 'fake',
 'fake',
 'small',
 'business',
 'Los',
 'Alamitos',
 'receive',
 'look',
 'like',
 'legitimate',
 'bill',
 '$',
 '70',
 'account',
 'number',
 ' ',
 'call',
 'phone',
 'number',
 'list',
 '866',
 '273',
 '7934',
 ' ',
 'wait',
 'time',
 'hold',
 'say',
 '20',
 'minute',
 'leave',
 'message',
 ' ',
 'live',
 'person',
 'phone',
 'matter',
 'number',
 'select',
 ' ',
 'leave',
 'firm',
 'message',
 'contact',
 'BBB',
 'attorney',
 'company',
 'try',
 'scam',
 'business',
 'illegal']

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [31]:
# creating a TF-IDF feature matrix

from sklearn.feature_extraction.text import TfidfVectorizer

def tfdif_vector(docs):
    # instantiate vectorizer

    tfdf = TfidfVectorizer(stop_words = 'english',
                           ngram_range = (1,2),
                           min_df = 3,
                           max_df = 0.5)

    # create a vocabulary and tf-idf score per document

    tf_score = tfdf.fit_transform(docs)

    # get feature names to use as dataframe column headers

    tf_score = pd.DataFrame(tf_score.todense(), 
                            columns = tfdf.get_feature_names())
    
    shape = print(f'tf_score shape = {tf_score.shape}')
    
    head = tf_score.head()
    
    return shape, head, tf_score, tfdf

In [23]:
# vectorize the text column from yelp dataset. 

# testing to see if the function works

tfdif_vector(yelp['text'])

tf_score shape = (10000, 31696)


(None,
     00  00 extra  00 pm  000  00am  00pm   01   04   05   06  ...  zucchini  \
 0  0.0       0.0    0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0  ...       0.0   
 1  0.0       0.0    0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0  ...       0.0   
 2  0.0       0.0    0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0  ...       0.0   
 3  0.0       0.0    0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0  ...       0.0   
 4  0.0       0.0    0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0  ...       0.0   
 
    zucchini fries  zumba   ça  équipe  érable  était  était très  été  être  
 0             0.0    0.0  0.0     0.0     0.0    0.0         0.0  0.0   0.0  
 1             0.0    0.0  0.0     0.0     0.0    0.0         0.0  0.0   0.0  
 2             0.0    0.0  0.0     0.0     0.0    0.0         0.0  0.0   0.0  
 3             0.0    0.0  0.0     0.0     0.0    0.0         0.0  0.0   0.0  
 4             0.0    0.0  0.0     0.0     0.0    0.0         0.0  0.0   0.0  
 
 [5 rows x 31696 columns],
        

In [32]:
# using the function above but this time assigning variables to it

tf_shape, tf_head, doc_vector, tfdf = tfdif_vector(yelp['text'])

tf_score shape = (10000, 31696)


In [33]:
# verifying that tfdf is a standalone object that can be called.

tfdf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.5, max_features=None,
                min_df=3, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [28]:
print(doc_vector.shape)
doc_vector.head()

(10000, 31696)


,00,00 extra,00 pm,000,00am,00pm,01,04,05,06,...,zucchini,zucchini fries,zumba,ça,équipe,érable,était,était très,été,être
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# creating a nearest neighbor model. 

from sklearn.neighbors import NearestNeighbors

# fit on vectorized text

nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')

nn.fit(doc_vector)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [30]:
# sample review. This is a real review from yelp

review = ["""

My boyfriend and I went here for the 
first time and I wouldn't be surprised 
if we went back today. It's that good. 
The tea selection is vast and they all 
looked quite tasty! They have milk-teas, 
regular teas, boba-teas... you name it, 
they got it! It seemed to be made to order, 
so everything was fresh!
They also have a large selection of cute 
snacks, cakes, and mochis. It all looked 
so tasty and fun. I got a chocolate mousse 
that is shaped like an ADORABLE dog and not 
only did it taste delicious, it was really 
really cool looking.
Everything is moderately priced and it is
an incredible experience all around.
The store has an awesome, open, and airy 
ambiance. When COVID restrictions are over, 
I would totally come here to do work. 
The store is clean and they are practicing 
social distancing like champs.
I would highly suggest coming here ASAP. 
It's almost too good to be true.
The cups they serve the drinks in are pretty epic. 
They have a cool top that doesn't allow for the drink to spill.

"""]

In [34]:
# query of similar document to the one posted above. 

# first, vectorize the sample review above. 

query = tfdf.transform(review)

In [45]:
# use nearest neighbor to find documents in the original yelp
# review that is similar to the review above. 

# nearest neighbor doesn't work on sparse matrix which is what query is
# first, convert the spart matrix to dense. 

dense = query.todense()

similar = nn.kneighbors(dense, return_distance=False)

In [53]:
# now print the documents from the original yelp review that is similar
# to the sample review above. 

# these are the top 10 reviews that are similar to the sample review above

similar.T

array([[6311],
       [6204],
       [7164],
       [1754],
       [4295],
       [2443],
       [9909],
       [5305],
       [6987],
       [2862]], dtype=int64)

In [56]:
# The third prediction makes perfect sense because this review 
# i grabbed from yelp was actually of a boba tea shop. INTERESTING. 

# The first 2 reviews in the prediction was in a different language. 
# That's interesting too. 

yelp['text'][7164]

"This place has really good boba! They offer a lot of flavors for drinks and they don't use artificial flavoring when it comes to the fruit smoothies. You can add more than just boba in your drinks and the lady at the front always gives suggestions on what you should order based on your preference. The boba has a good consistency and it's not over cooked. Their ice teas are really good and the milk teas as well. Overall a really good place to get boba drinks."

In [65]:
# All predicted similar reviews

print(yelp['text'][6311], "\n\n", 
      yelp['text'][6204], "\n\n",
      yelp['text'][7164], "\n\n",
      yelp['text'][1745], "\n\n",
      yelp['text'][4295], "\n\n",
      yelp['text'][2443], "\n\n",
      yelp['text'][9909], "\n\n",
      yelp['text'][5305], "\n\n",
      yelp['text'][6987], "\n\n",
      yelp['text'][2862])

天氣很熱吃不下東西，今天我點了一個韓國冷面湯、餐後點了甜點，冰沙系列不會太甜膩，覺得店家很用心製作，包含擺盤精緻、佐料衛生，夏日想開胃，這是一個不錯的選擇，服務人員也很敬業，以後會常常來 

 旅行でラスベガスに来ましたがネイルがはげてるのが気になり、探したお店でした。
質問にも丁寧に答えてくれましたし、日本人の方も日本語が話せる方も居て、とても綺麗で居心地のいいお店でした。 
ネイルはちはるさんと言う綺麗な方が丁寧にしてくれとても気に入りました。
予定になかったまつ毛エクステもお願いし、日本ではまだあまりないブラウンカラーのエクステをしてもらい、とても気に入りました。
また是非マッサージなどで伺いたいと思います。 

 This place has really good boba! They offer a lot of flavors for drinks and they don't use artificial flavoring when it comes to the fruit smoothies. You can add more than just boba in your drinks and the lady at the front always gives suggestions on what you should order based on your preference. The boba has a good consistency and it's not over cooked. Their ice teas are really good and the milk teas as well. Overall a really good place to get boba drinks. 

 A ok for sure!! Pretty decent Mexican food in a cutesy little area. The lunch combos were a bit high for this type of joint, around $9. 

The chips and salsa were great and service rwas friendly but slow. 

Good solid c

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [66]:
## Your code 

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [67]:
# instantiate random forest classifier

random = RandomForestClassifier()

In [68]:
# define the pipeline
# use the vectorizer already defined in the tfdif vector function

pipe = Pipeline([
    ('vect', tfdf),
    ('clf', random)
])

In [70]:
# split into train and test dataset. 

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(yelp['text'], 
                                                    yelp['stars'], 
                                                    test_size=0.2, 
                                                    stratify=yelp['stars'],
                                                    random_state=42)

In [71]:
# fit the pipeline to the training dataset. 

pipe.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.5, max_features=None,
                                 min_df=3, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_patte...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None

In [108]:
from sklearn.metrics import accuracy_score

# accuracy score

y_pred = pipe.predict(x_test)
accuracy_score(y_test, y_pred)

0.562

In [72]:
# using the pipeline above to predict star rating for sample review

# our pipeline gives us a prediction of 4. It was actually 5. 

pipe.predict(review)

array([4], dtype=int64)

In [74]:
# use gridsearch cv to find the optimal parameters for our model

param = {
    'vect__max_df': (0.5, 1.0),
    'vect__min_df': (2, 5, 10),
    'vect__max_features': (500, 10000),
    'clf__n_estimators': (5, 20),
    'clf__max_depth': (10, 50, None)
}

grid_search = GridSearchCV(pipe, param, cv = 5, n_jobs=-1, verbose=1)
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   17.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  3.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=0.5,
                                                        max_features=None,
                                                        min_df=3,
                                                        ngram_range=(1, 2),
                                                        no

In [105]:
#best_scores from grid_search cf

grid_search.best_score_

0.570125

In [76]:
# best parameters from grid_search cv. 

grid_search.best_params_

{'clf__max_depth': None,
 'clf__n_estimators': 20,
 'vect__max_df': 1.0,
 'vect__max_features': 10000,
 'vect__min_df': 2}

In [109]:
# re-doing the prediction with the best parameters above. 

random2 = RandomForestClassifier(max_depth = None,
                               n_estimators = 20)

tfdf2 = TfidfVectorizer(stop_words = 'english',
                           ngram_range = (1,2),
                           min_df = 2,
                           max_features = 10000)


pipe = Pipeline([
    ('vect', tfdf2),
    ('clf', random2),
])

pipe.fit(x_train, y_train)

# our prediction using new parametetrs
print (f'New star prediction {pipe.predict(review)}')

# accuracy score
y_pred = pipe.predict(x_test)
accuracy_score(y_test, y_pred)

 new prediction [4]


0.5685

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [79]:
## Your code 

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd
from tqdm import tqdm
tqdm.pandas()

C:\Users\Samsung\Anaconda3\envs\unit4-NLP\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## This text document is 10,000 rows long. It will take a CENTURY to tokenize all of it. So, For this part of the assignment, I will use a sample of 25 rows/documents instead of the entire document. 

In [92]:
# sample dataset. 

df = yelp.sample(random_state = 42, n=25)

print(df.shape)
df.head()

(25, 9)


,business_id,cool,date,funny,review_id,stars,text,useful,user_id
6252,QE1tzFyyyRKV1p7C4LRukQ,0,2015-07-15 02:41:12,0,pcpmloUy9OSbQCxG-NIXKA,1,"We ordered 1/4 chicken leg and thigh take out,...",1,AnHGoMsQ62-cUh1pl4bG9g
4684,wghDrzcZ0VloAtaIZ7GEBg,0,2012-05-05 22:23:44,0,-Gb3R1WOoa_fPpsQ91rLCg,4,nice hike...keeps going up and up and up ...an...,0,7ziWZULyiZv2TesYNMFf4g
1731,78qDAXntkxbmbiMdfbUJiw,3,2010-10-01 22:04:55,0,uBlyX9pkMmp9evijZtuQ6g,3,I have nothing against trainee teachers.. but ...,2,CxDOIDnH8gp9KXzpBHJYXw
4742,MtjOk7u7sp3yWyzscAIG4Q,0,2013-06-14 01:56:44,0,fHwnCTCeXE5HWdTvGR1LZg,4,My boyfriend and I went this evening for dinne...,0,bVBLGgj0TMKoSRtcZv6_Bw
4521,KTPRYqiFdLowAUEAnN7e3g,1,2013-02-23 23:04:31,0,ZlUvD0BCVNoB4QHarlTcYA,2,Waitresses dressed like Hooters servers. Burge...,3,TSFRIumlH-tq0hKh9YyhVg


In [93]:
# adding a column to yelp reviews that has the tokens

df['tokens'] = df['text'].progress_apply(tokenize)

df.head(1)

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [11:22<00:00, 27.30s/it]


,business_id,cool,date,funny,review_id,stars,text,useful,user_id,tokens
6252,QE1tzFyyyRKV1p7C4LRukQ,0,2015-07-15 02:41:12,0,pcpmloUy9OSbQCxG-NIXKA,1,"We ordered 1/4 chicken leg and thigh take out,...",1,AnHGoMsQ62-cUh1pl4bG9g,"[order, 1/4, chicken, leg, thigh, totally, bur..."


In [94]:
# creating a dictionary representation of all the words

dct = corpora.Dictionary(df['tokens'])

In [95]:
# bag of words representation of the corpus

corpus = [dct.doc2bow(text) for text in df['tokens']]

In [97]:
# applying LDA to document. 

lda = LdaMulticore(corpus=corpus,
                  id2word = dct,
                  random_state = 42,
                  num_topics=5,
                  passes=10)

In [99]:
# making the resulting topics look pretty 

import re

words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:6]) for t in words]

for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
  service food 
 noise part

------ Topic 1 ------


 place leak start good get

------ Topic 2 ------


 good like   try customer

------ Topic 3 ------
order special restaurant tofu minimum food

------ Topic 4 ------
  great time service dress good



In [100]:
# creating visual interpretation of LDA topics

import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [101]:
pyLDAvis.gensim.prepare(lda, corpus, dct)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.052502 -0.127984       1        1  37.023750
1     -0.143803  0.014704       2        1  25.468954
0      0.086450  0.100517       3        1  23.649954
4      0.015194  0.010817       4        1  11.192327
3     -0.010344  0.001946       5        1   2.665019, topic_info=           Term       Freq      Total Category  logprob  loglift
0                15.000000  15.000000  Default  30.0000  30.0000
151     special   4.000000   4.000000  Default  29.0000  29.0000
150  restaurant   5.000000   5.000000  Default  28.0000  28.0000
6         order   5.000000   5.000000  Default  27.0000  27.0000
101        food   7.000000   7.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
101        food   0.249042   7.250254   Topic5  -5.0243   0.2538
60        great   0.247573   9.458508   Topic5  -5.0302  -0.0180
602        live   0.041953   1.267917   Topic5  -6.8054   0.2164
580       Hakka   0.042868   1.956399   Topic5  -6.7838  -0.1958
10         \n\n   0.046524  23.217325   Topic5  -6.7019  -2.5877

[264 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
160       1  0.130966     \n
160       2  0.392897     \n
160       3  0.523863     \n
10        1  0.473784   \n\n
10        2  0.387642   \n\n
...     ...       ...    ...
493       2  0.247037   week
493       3  0.741111   week
721       1  0.897106   wine
708       3  1.020365   work
445       1  0.688364  wrong

[278 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 5, 4])

The visualization above shows us that there are 4 main topics because there is some overlab going on
between topic 4 and topic 5. There are 4 main sections which tells us that there probably are 4 main topics. 

Should probably retrain LDA model with 4 topics to avoid topic overlap. 

Topic 1 represent words that appear most often in almost all the documents. 
Topics 4 and 5 represent words that appear least often in almost all the documents. 

The red bar represent the appearance of a particular word in the selected topic vs
blue bar which represent the occurane of the same word in the entire document. 

red bar = estimated word frequency withing topic

blue bar = frequency of word in entire document. 

Sliding the relevance metric to 0 shows numbers the words that are 
relevant to just the topic selected
but doesn't appear in other topics. 

Sliding the relevance metric to any number below 1 focuses in on words 
relevent to that particular topic, could be present in other topics, but more rare. 

So, the relevance metric can be used to 
* find words that are specific to a particular topic,
* figure out words that appear most often in a particular topic, 
* figure out which words appear most often in the entire document, 
* figure out how many unique topics there are, and many other things. 


## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)

In [ ]:
## Your code